# Cohort Project - Supplemental - $H_2$ Ising Hamiltonian solver

> An attempt at implementing a $H_2$ ground state calculator leveraging the paper [arXiv:1611.01068v1 [quant-ph] 3 Nov 2016](https://arxiv.org/pdf/1611.01068.pdf)






## Summary

Authors of the [paper](https://arxiv.org/pdf/1611.01068.pdf) have provided a method to express the calculation of the $H_2$ molecule ground state energy as a two-body Ising Hamiltonian. 

The DWave Quantum Annealer is designed for solving Ising Hamiltonians. 

This implementation attempts to follow the recipe laid out in Supplemental Material (page 6 - para. 1 Detailed Procedure ) which refer to the  formulas (4), (5), (6) and (7). Table I (page 7) is represented in the data file H2_coefficients_exact_simulated.csv which we use to compute the simulated energy to compare to the exact energy, and thus validate the computation.


# Experimenting for $H_2$ Energy Calculations

## Implementing Hamiltonian for calculating $H_2$ Energies








### Outcome

At this stage the results do not match the target exact numbers provided. The code will need reviewing to verify the application of the recipe. 

One peculiar outcome is that results obtained for row N strangely coincide with the target on row N+1. If the code is considered correct, one might need to have the data Table I in the paper validated to ensure the energies indicated are indeed associated with the correct coefficients for the given molecular distance on each row.

See results under the "Process" header further below.


## Procedure


### Excerpt from the paper

We can use this symmetry to reduce the
Hamiltonian to the following effective Hamiltonian, acting only on two qubits:

>(4) $H_{H_2}=g_01+g_1σ_{z}^{0}+g_2σ_{z}^{1}+g_3σ_{z}^{0}σ_{z}^{1}+g_4σ_{x}^{0}σ_{x}^{1}+g_4σ_{y}^{0}σ_{y}^{1}$ 
>$=g_01+H_0$    

>(5) $H_0=g_1σ_{z}^{0}+g_2σ_{z}^{1}+g_3σ_{z}^{0}σ_{z}^{1}+g_4σ_{x}^{0}σ_{x}^{1}+g_4σ_{y}^{0}σ_{y}^{1}$         

By squaring the Hamiltonian $H_0$ and modifying it, one can get a new Ising Hamiltonian:

>(6) $H_1=H_{0}^2 + 2g_3H_0=a_1+a_2(σ_{z}^{0}+σ_{z}^{1}) +a_3σ_{z}^{0}σ_{z}^{1}$

With:  

>(7) 

>$a_1=g_1^2+g_2^2+g_3^2+ 2g_4^2$ 

>$a_2 = 2(g_1+g_2)g_3$ 

>$a_3= 2(g_1g_2−g_4^2+g_3^2)$



Here we present steps to get the ground state of $H_{H_2}$ by using the new Ising Hamiltonian H1 (Eq.6).
1. If $|g_1| + |g_2| + |g_4| < |g_3|$ start computing by $H_1$ and get the result $Y$ . Otherwise increase $|g_3|$ by
$|g_1| + |g_2| + |g_4|$ and start computing.
2. Solve equation $x2 + 2g3x = Y$ and get $σ_x^1$ and $σ_x^2$  $(σ_x^1<= σ_x^2)$. Add $|g_1| + |g_2| + |g_4|$ to $σ_x^1$
 if added to $g_3$ before (we just assume $g_3 > 0$.) Compare $σ_x^1$ with $g_3 − g_1 − g_2$ (or $g_3 + g_1 + g_2$) to get the ground state of $H_0$. Add $g_0$ to get the ground state of $H_{H_2}$.


# Code

## Toolkit installation

In [ ]:
!pip install dwave-ocean-sdk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Make the dwave data set folder the current directory
# and provide path to the coefficients file

h2coef_path = "/content/drive/My Drive/Colab Notebooks/"
%cd $h2coef_path
#!ls


/content/drive/My Drive/Colab Notebooks


In [ ]:
import numpy as np
import csv 

## DWave library check

Note: Libraries used do not actually run on the QPU but uses Tabu sampler library


In [ ]:
import networkx as nx
from collections import defaultdict
from dimod import BinaryQuadraticModel
from tabu import TabuSampler
import dimod

In [ ]:
# Sanity check using TabuSampler (not on the QPU)

response = TabuSampler().sample_ising({'a': -0.5, 'b': 1.0}, {('a', 'b'): -1})

print(response)




   a  b energy num_oc.
0 -1 -1   -1.5       1
['SPIN', 1 rows, 1 samples, 2 variables]


In [ ]:
Q = defaultdict(int)
sampler = TabuSampler()
offset = 0

# Solve y = -5 xa -3 xb -8 xc -6 xd + 4 xaxb + 8 xaxc + 2 xbxc + 10 xcxd
# Answer should be -11 for qubits 1, 0, 0, 1

Q = [(-6, 5, 0, 0), (5,-8,1,4), (0,1,-3,2), (0,4,2,-5)]

bqm = BinaryQuadraticModel.from_qubo(Q, offset=offset)
sampleset = sampler.sample(bqm)
print(sampleset)
print(sampleset.first.energy)


   0  1  2  3 energy num_oc.
0  1  0  0  1  -11.0       1
['BINARY', 1 rows, 1 samples, 4 variables]
-11.0


## Class for $H_2$ energy calculation

Loads the table file included in the source paper.

Calculates the ground state energy given one set of coefficients (one row)
from the table

In [ ]:


class H2_Energy:

  def data(self):
    return self.coeff

  # Initialize with coefficient data
  def __init__(self,file):

    self.coeff = []
    with open(file, newline='') as csvfile:
      reader = csv.reader(csvfile, delimiter=',')
      first = True
      for row in reader:
        if ( first ): 
          first = False
          continue
        row = list(map(float, row)) 
        self.coeff.append(row)


  def solve_ising(self,R,g0,g1,g2,g3,g4,samples,exact,verbose):
    return self.solve(R,g0,g1,g2,g3,g4,False,samples,exact,verbose)

  def solve_qubo(self,R,g0,g1,g2,g3,g4,samples,exact,verbose):
    return self.solve(R,g0,g1,g2,g3,g4,True,samples,exact,verbose)

  # Solve the energy for the given radius and coefficients

  def solve(self,R,g0,g1,g2,g3,g4,qubo,samples,exact,verbose):

    use_QUBO = qubo

    print("\nSolving for R=%f (exact=%f)" % (R,exact),g0,g1,g2,g3,g4) 

    # Step 1: If |g1| + |g2| + |g4| >= |g3| : g3 += |g1| + |g2| + |g4|
    # Remember the g3 addition for later

    g3_addition = 0
    if ( abs(g1) + abs(g2) + abs(g4) >= abs(g3)) :
      g3_addition = (abs(g1)+abs(g2)+abs(g4))
      # g3 = abs(g3) + g3_addition
      g3 = (abs(g3) + g3_addition)


    # Step 1b: calculate a1, a2, a3
    # a1 = g1**2 + g2**2 + g3**2 + 2 * g4**2
    # a2 = 2 * ( g1 + g2) * g3
    # a3 = 2 * (g1*g2 - g4**2 + g3**2)

    a1 = g1**2 + g2**2 + g3**2 + 2 * g4**2
    a2 = 2 * ( g1 + g2) * g3
    a3 = 2 * (g1*g2 - g4**2 + g3**2)

    # Step 2 : Solve H0**2 + 2g3H0 = a1 + a2( sz0 + sz1 ) + a3 (sz0*sz1) = Y
    # (the squared H0 hamiltonian)
    # where sz0 is sigma z bit 0, ...
    # Convert szi -> (2 xi - 1) (ising to qubo conversion)
    # then simplify the equation to obtain the quadratic coeeficients of a two qubit system
    # Calculated Q matrix:
    #   |  (2a2-2a3)    2a3     |
    #   |     2a3    (2a2-2a3)  |

    c_a = 2 * a2 - 2 * a3
    c_b = 4 * a3



    #print(Q)

    # Solve the equation. First solution is lowest energy
    # Using BINARY 

    if ( use_QUBO ):

      Q=defaultdict(float)
      Q[0,0] = c_a
      Q[0,1] = c_b
      Q[1,0] = c_b
      Q[1,1] = c_a 

      #Q = [(2 * a2 - 2 * a3, 2 * a3),(2 * a3,2 * a2 - 2 * a3 )]

      offset = 0
      bqm = BinaryQuadraticModel.from_qubo(Q, offset=offset)
      sampleset = sampler.sample(bqm, num_reads = samples)
      #print(sampleset.first.sample.keys() )
      #print(sampleset.first.energy)
      if ( verbose==True ): print(sampleset.first.sample)
      if (verbose==True): print(sampleset)

      # Step 3: Get x0 and x1 for first energy result

      for set in sampleset.data():
        x0 = set.sample[0]
        x1 = set.sample[1]
        energy = set.energy
        if (verbose==True): print("x0,x1,ener : ",x0,x1,energy)
        break

      Y = 4 * x0 * x1 + (2*a2 - 2*a3) * x0 + (2*a2-2*a3) * x1 + a3 - 2*a2 + a1

      # convert x0,x1 to ising spins 

      sz0 = ( 2 * x0 ) -1 
      sz1 = ( 2 * x1 ) -1 

    else:

      # Using SPIN (ising)
      # H = h_1 * s_1 + h_2 * s_2 + J_{1,2} * s_1 *s_2
      response = TabuSampler().sample_ising({'a': c_a, 'b': c_a}, {('a', 'b'): c_b}, num_reads = samples)
      
      if ( verbose==True): print(response)

      for set in response.data():
        sz0 = set.sample['a']
        sz1 = set.sample['b']
        energy = set.energy
        if (verbose==True): print("sz0,sz1,ener : ",sz0,sz1,energy)
        break

      # Step 4: Calculate Y = ( a1 + a2( sz0 + sz1 ) + a3 (sz0*sz1))

      Y = ( a1 + a2 * ( sz0 + sz1 ) + a3 * (sz0 * sz1))

      # Convert to get x0,x1

      x0 = (sz0 + 1) / 2
      x1 = (sz1 + 1) / 2
    
      
    # Get hx1 and hx2 in :
    # hx**2 + 2*g3*hx = Y
    # a = 1, b = 2g3, c = -Y

    a = 1
    b = 2 * g3
    c = -Y

    # Solve H1 for x (minimum of the two possibilities)

    hx1 = ( -b + np.sqrt(b**2-4*a*c)) / (2 * a ) 
    hx2 = ( -b - np.sqrt(b**2-4*a*c)) / (2 * a ) 

    #print("hx1,hx2",hx1,hx2)

    if ( hx2 < hx1):
      swp = hx2
      hx2 = hx1
      hx1 = swp

    # Add g3_addition to hx1

    hx1 += g3_addition

    # H0 = g1σ0 z + g2σ1 z + g3σ0 zσ1 z +
  
    H0_ver = (g1 * sz0) + (g2 * sz1) + (g3 * sz0 * sz1) 

    H0 = hx1

    H = H0 + g0

    return (H)



H2 = H2_Energy( h2coef_path +'H2_coefficients_exact_simulated.csv')
print(H2.data())

#'H2_coefficients_exact_simulated.csv'


[[0.6, 1.5943, 0.5132, -1.1008, 0.6598, 0.0809, -0.5703, -0.5703], [0.65, 1.4193, 0.5009, -1.0366, 0.6548, 0.0813, -0.6877, -0.6877], [0.7, 1.2668, 0.4887, -0.9767, 0.6496, 0.0818, -0.7817, -0.7817], [0.75, 1.1329, 0.4767, -0.9208, 0.6444, 0.0824, -0.8575, -0.8575], [0.8, 1.0144, 0.465, -0.8685, 0.639, 0.0829, -0.9188, -0.9188], [0.85, 0.909, 0.4535, -0.8197, 0.6336, 0.0835, -0.9685, -0.9685], [0.9, 0.8146, 0.4422, -0.774, 0.6282, 0.084, -1.0088, -1.0088], [0.95, 0.7297, 0.4313, -0.7312, 0.6227, 0.0846, -1.0415, -1.0415], [1.0, 0.6531, 0.4207, -0.691, 0.6172, 0.0852, -1.0678, -1.0678], [1.05, 0.5836, 0.4103, -0.6533, 0.6117, 0.0859, -1.0889, -1.0889], [1.1, 0.5204, 0.4003, -0.6178, 0.6061, 0.0865, -1.1056, -1.1056], [1.15, 0.4626, 0.3906, -0.5843, 0.6006, 0.0872, -1.1186, -1.1186], [1.2, 0.4098, 0.3811, -0.5528, 0.5951, 0.0879, -1.1285, -1.1285], [1.25, 0.3613, 0.372, -0.523, 0.5897, 0.0886, -1.1358, -1.1358], [1.3, 0.3167, 0.3631, -0.4949, 0.5842, 0.0893, -1.1409, -1.1409], [1.35, 0.2

## Process the table and calculate the $H_2$ ground states


In [ ]:
data = H2.data()

samples = 50
for row in range(len(data)):
  R,g0,g1,g2,g3,g4,exact,sim = data[row]
  #print(R,g0,g1,g2,g3,g4)
  HH2_i = H2.solve_ising(R,g0,g1,g2,g3,g4, samples, exact, False )
  print("HH2 ising = %f, Exact = %f" % (HH2_i,exact))
  HH2_q = H2.solve_qubo(R,g0,g1,g2,g3,g4, samples, exact, False )
  print("HH2 qubo = %f, Exact = %f" % (HH2_i,exact))



Solving for R=0.600000 (exact=-0.570300) 1.5943 0.5132 -1.1008 0.6598 0.0809
HH2 ising = -0.687590, Exact = -0.570300

Solving for R=0.600000 (exact=-0.570300) 1.5943 0.5132 -1.1008 0.6598 0.0809
HH2 qubo = -0.687590, Exact = -0.570300

Solving for R=0.650000 (exact=-0.687700) 1.4193 0.5009 -1.0366 0.6548 0.0813
HH2 ising = -0.781574, Exact = -0.687700

Solving for R=0.650000 (exact=-0.687700) 1.4193 0.5009 -1.0366 0.6548 0.0813
HH2 qubo = -0.781574, Exact = -0.687700

Solving for R=0.700000 (exact=-0.781700) 1.2668 0.4887 -0.9767 0.6496 0.0818
HH2 ising = -0.857304, Exact = -0.781700

Solving for R=0.700000 (exact=-0.781700) 1.2668 0.4887 -0.9767 0.6496 0.0818
HH2 qubo = -0.857304, Exact = -0.781700

Solving for R=0.750000 (exact=-0.857500) 1.1329 0.4767 -0.9208 0.6444 0.0824
HH2 ising = -0.918683, Exact = -0.857500

Solving for R=0.750000 (exact=-0.857500) 1.1329 0.4767 -0.9208 0.6444 0.0824
HH2 qubo = -0.918683, Exact = -0.857500

Solving for R=0.800000 (exact=-0.918800) 1.0144 0.4